In [26]:
import numpy as np
import numpy.ma as ma
import datetime

def thresholding(image,th=128):
    image = np.array(image)
    image = image>th
    return image*255

def thinning(image,th=3):
    image = np.array(image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            sum = np.sum(image[i,j:j+th])
            if(sum==th*255):
                image[i,j]=0
                j = j+th-1
    return image

def fetchNeighbours(image,i,j):
    neighbours = [0,0,0,0,0,0,0,0,0,0]
    dim = np.shape(image)
    if(i>0 and i<dim[0]-1 and j>0 and j<dim[1]-1):
        neighbours[2]=image[i-1,j]
        neighbours[3]=image[i-1,j+1]
        neighbours[4]=image[i,j+1]
        neighbours[5]=image[i+1,j+1]
        neighbours[6]=image[i+1,j]
        neighbours[7]=image[i+1,j-1]
        neighbours[8]=image[i,j-1]
        neighbours[9]=image[i-1,j-1]
    elif(i==0 and j>0 and j<dim[1]-1):
        neighbours[4]=image[i,j+1]
        neighbours[5]=image[i+1,j+1]
        neighbours[6]=image[i+1,j]
        neighbours[7]=image[i+1,j-1]
        neighbours[8]=image[i,j-1]
    elif(i==dim[0]-1 and j>0 and j<dim[1]-1):
        neighbours[2]=image[i-1,j]
        neighbours[3]=image[i-1,j+1]
        neighbours[4]=image[i,j+1]
        neighbours[8]=image[i,j-1]
        neighbours[9]=image[i-1,j-1]
    elif(i>0 and i<dim[0]-1 and j==0):
        neighbours[2]=image[i-1,j]
        neighbours[3]=image[i-1,j+1]
        neighbours[4]=image[i,j+1]
        neighbours[5]=image[i+1,j+1]
        neighbours[6]=image[i+1,j]
    elif(i>0 and i<dim[0]-1 and j==dim[1]-1):
        neighbours[2]=image[i-1,j]
        neighbours[6]=image[i+1,j]
        neighbours[7]=image[i+1,j-1]
        neighbours[8]=image[i,j-1]
        neighbours[9]=image[i-1,j-1]
    elif(i==0 and j==0):
        neighbours[4]=image[i,j+1]
        neighbours[5]=image[i+1,j+1]
        neighbours[6]=image[i+1,j]
    elif(i==0 and j==dim[1]-1):
        neighbours[6]=image[i+1,j]
        neighbours[7]=image[i+1,j-1]
        neighbours[8]=image[i,j-1]
    elif(i==dim[0]-1 and j==0):
        neighbours[2]=image[i-1,j]
        neighbours[3]=image[i-1,j+1]
        neighbours[4]=image[i,j+1]
    elif(i==dim[0]-1 and j==dim[1]-1):
        neighbours[8]=image[i-1,j]
        neighbours[9]=image[i,j-1]
        neighbours[2]=image[i-1,j-1]
    return neighbours

def thin(image,ite):
    image = np.array(image)
    markedPixels= []
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            neighbours = fetchNeighbours(image,i,j)
            print(neighbours)
            nonZeroNeighboursCount = np.sum(neighbours)/255
            transitionCount = 0
            for k in range(len(neighbours)-1):
                if(neighbours[k]!=neighbours[k+1]):
                    transitionCount+=1
            mark = (image[i,j]>0 and nonZeroNeighboursCount in [2,3,4,5,6] and transitionCount==1)
            #mark = mark and neighbours[5]!=0
            if(ite==1):
                mark = mark and (neighbours[2]*neighbours[4]*neighbours[6])==0 and (neighbours[4]*neighbours[6]*neighbours[8])==0
            else:    
                mark = mark and (neighbours[2]*neighbours[4]*neighbours[8])==0 and (neighbours[2]*neighbours[6]*neighbours[8])==0
            if(mark):
                image[i,j]=0
    return image
           
def mass(image):
    image = np.array(image)
    return np.sum(image)

def averagemass(image):
    m = mass(image)
    return m/image.size

def xmass(image):
    image = np.array(image)
    dim = np.shape(image)
    mid = dim[1]/2
    leftmass = 0
    rightmass = 0
    for j in range(dim[1]):
        if(j<mid):
            leftmass = leftmass + np.sum(image[:,j])
        else:
            rightmass = rightmass + np.sum(image[:,j])
    return (leftmass,rightmass)

def averagexmass(image):
    xm = xmass(image)
    return (xm[0]/image.size,xm[1]/image.size)


def ymass(image):
    image = np.array(image)
    dim = np.shape(image)
    mid = dim[0]/2
    topmass = 0
    bottommass = 0
    for i in range(dim[0]):
        if(i<mid):
            topmass = topmass + np.sum(image[i,:])
        else:
            bottommass = bottommass + np.sum(image[i,:])
    return (topmass,bottommass)

def averageymass(image):
    xm = ymass(image)
    return (xm[0]/image.size,xm[1]/image.size)


def boundarysquare(image):
    image = np.array(image)
    dim = np.shape(image)
    topRow = leftColumn = 0
    bottomRow = dim[0]
    rightColumn = dim[1]
    topRowFound = bottomRowFound = leftColumnFound = rightColumnFound = False
    
    for i in range(dim[0]):
        s = np.sum(image[i,:])
        if(s>0 and not topRowFound):
            topRow = i
            topRowFound = True
        if(s==0 and topRowFound and not bottomRowFound):
            bottomRow = i
            bottomRowFound = True
        
    for j in range(dim[1]):
        s = np.sum(image[:,j])
        if(s>0 and not leftColumnFound):
            leftColumn = j
            leftColumnFound = True
        if(s==0 and leftColumnFound and not rightColumnFound):
            rightColumn = j
            rightColumnFound = True
    
    return image[topRow:bottomRow,leftColumn:rightColumn]



'''
img = thresholding([[1,2,3,3,3,1],[1,55,55,55,1,1],[1,55,55,55,55,1],[1,3,1,1,1,1],[1,3,1,1,1,1]],3)
print(img)
print(mass(img))
print(averagemass(img))
print(img.size)
print(xmass(img))
print(averagexmass(img))
print(ymass(img))
print(averageymass(img))
print(boundarysquare(img))

#print(fetchNeighbours(img,0,4))
img=thin(img,1)
print(img)
print(thin(img,2))
'''


def avgHorizontalStroke(image):
    dim = np.shape(image)
    stroke = 0
    for i in range(dim[0]):
        strokeSize = []
        count = 0
        for j in range(dim[1]):
            if(image[i,j]>0):
                count += 1
            else:
                strokeSize.append(count)
                count=0
        stroke = np.max(strokeSize)*(i+1)/dim[1]
    return stroke

print("start : "+ str(datetime.datetime.now()))
print("end : "+ str(datetime.datetime.now()))

a = [[0,0,0,0],[1,1,1,1]]
a = np.array(a)
#print(np.power(a,-1/2))
#a=np.linalg.inv(a)
#print(np.sqrt(a))

cov = np.cov(a.T)
eigValues,eigVectors = np.linalg.eig(cov)
print(eigValues)
#print(np.diag(eigValues))
#print(eigVectors)
#print(cov)

s = [5,5]
s = np.array(s)


start : 2016-12-30 16:36:19.512949
end : 2016-12-30 16:36:19.513094
[ -2.22044605e-16   2.00000000e+00   0.00000000e+00   0.00000000e+00]
